In [611]:
# import necessary python modules
import csv
import pandas as pd
import numpy as np
from __future__ import division 

# Create new data frames and import raw data

Downloaded Excel files were re-saved as CSV. The following lines import those files for each state and create our four primary tables with the raw data from BLM.

In [466]:
# create a list of the states we'll be analyzing, will be used frequently
statelist = ["AZ", "CA", "CO", "ID", "MT", "NM", "NV", "OR", "UT", "WY"]

In [467]:
# Create a new empty DataFrame for each table
allotments = pd.DataFrame()
operators = pd.DataFrame()
permits = pd.DataFrame()
health = pd.DataFrame()

#create a variable storing the path for each set of raw files
allotments_path = 'data/rangeland-administration-system/allotment-info/'
operators_path = 'data/rangeland-administration-system/operator-info/'
permits_path = 'data/rangeland-administration-system/permit-schedule-info/'
health_path = 'data/rangeland-health/'

# Import those files for allotments ... (probably should be a function)
for s in statelist:
    csv_file = '{}{}.csv'.format(allotments_path, s)
    new_data = pd.read_csv(csv_file, dtype={'Allotment Number': 'object', 'Auth No': 'object'})
    allotments = allotments.append(new_data)

# ... and for operators
for s in statelist:
    csv_file = '{}{}.csv'.format(operators_path, s)
    new_data = pd.read_csv(csv_file, dtype={'Allotment Number': 'object', 'Auth No': 'object'})
    operators = operators.append(new_data)

# ... and for permits
for s in statelist:
    csv_file = '{}{}.csv'.format(permits_path, s)
    new_data = pd.read_csv(csv_file, dtype={'Allotment Number': 'object', 'Auth No': 'object'})
    permits = permits.append(new_data)

# ... and for land health standards
for s in statelist:
    csv_file = '{}{}.csv'.format(health_path, s)
    new_data = pd.read_csv(csv_file, dtype={'Allotment Number': 'object', 'Auth No': 'object'})
    health = health.append(new_data)

# Processing for Django

###  Field Offices table
---

The following lines of code creates a table of field offices by dropping duplicates and adding a unique ID as an integer. This ID can be referenced by other tables in Django.


In [468]:
# create a new df with field office info
field_offices = allotments[["Admin Office", "Field Office"]]

In [469]:
#  select only uniques from the DB
field_offices.drop_duplicates('Admin Office', inplace = True)

In [470]:
# save and re-read as a CSV for a janky but fast way of generating unique IDs starting at 1
field_offices.to_csv('data/processed/field_offices.csv')
field_offices = pd.read_csv('data/processed/field_offices.csv')

In [471]:
# Create a new column for field office state.
field_offices["State"] = field_offices["Admin Office"]
field_offices["State"] = field_offices["State"].str[2:4]

In [472]:
field_offices.loc[field_offices['State'] == 'AZ', 'StateCode'] = 1
field_offices.loc[field_offices['State'] == 'CA', 'StateCode'] = 2
field_offices.loc[field_offices['State'] == 'CO', 'StateCode'] = 3
field_offices.loc[field_offices['State'] == 'ID', 'StateCode'] = 4
field_offices.loc[field_offices['State'] == 'MT', 'StateCode'] = 5
field_offices.loc[field_offices['State'] == 'NM', 'StateCode'] = 6
field_offices.loc[field_offices['State'] == 'NV', 'StateCode'] = 7
field_offices.loc[field_offices['State'] == 'OR', 'StateCode'] = 8
field_offices.loc[field_offices['State'] == 'UT', 'StateCode'] = 9
field_offices.loc[field_offices['State'] == 'WY', 'StateCode'] = 10

field_offices['StateCode'] = field_offices['StateCode'].astype('int64')

In [473]:
field_offices['id'] = field_offices.index
field_offices['id'] = field_offices['id'] + 1

In [474]:
field_offices=field_offices.rename(columns = {'Admin Office':'office_code', 'Field Office': 'office_name', 'StateCode': 'state_id'})

In [475]:
field_offices = field_offices[['id', 'office_code', 'office_name', 'state_id']]
field_offices.to_csv("data/processed/field_offices.csv")

###Operators table
---

The following lines of code create a table of operators (ranches) with a unique ID that can be referenced by other tables, such as the permits table. Create a table of operators linked to field offices, and with a unique ID that can be referencecd by other tables

In [476]:
#rename operators columsn to remove spaces and capital
operators=operators.rename(columns = {'Off CD': 'office_code', 'Auth No':'auth_no', 'Operator Display Name': 'operator_display_name', 'Address1': 'address1', 'Address2': 'address2', 'City': 'city', 'Phone Number': 'phone_number', 'Release Text': 'release_text', 'Zipcode1 5': 'zipcode15', 'Zipcode6 9': 'zipcode69'})

In [477]:
#assign an id based on the index, but skip the 0
operators["id"] = operators.index + 1

In [478]:
#create a new dataframe that joins operators with field offices
new_ops = pd.merge(operators, field_offices, on='office_code', how='inner')

In [479]:
#that worked great now overwrite operators with that same data
operators = new_ops

In [480]:
#rename the two different id fields so we've got what we want.
operators['id'] = operators['id_x'].astype(int)
operators['field_office_id'] = operators['id_y'].astype(int)

In [481]:
# shed the data we don't want by reassigning the variable name 'operators' new a new dataframe with only these columns selected
operators = operators[['id', 'auth_no', 'operator_display_name', 'address1', 'address2', 'city', 'zipcode15', 'zipcode69', 'ST2', 'phone_number', 'release_text', 'field_office_id']]

In [482]:
# concatenate the ranch name and the city to create a unique identifier, just in case
# operator names appear to have no duplicates
operators['unique'] = operators['operator_display_name'] + operators['city']

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [679]:
# check the first 10 rows
operators['unique'].head()

0    NOEL,MICHAEL E,SHERRY M;STEWART,TREVOR D,JENNI...
1              DUANE E. AND IVONA H. MAGOONBUNKERVILLE
2                 LARSON, L. LYNN AND SANDRAST. GEORGE
3                     SUNSHINE CATTLE CO. LCST. GEORGE
4                       HEATON CATTLE COMPANYST GEORGE
Name: unique, dtype: object

In [484]:
# create a new datafraem and drop any duplicates, checking for duplicates using our new unique field
# operators_unique = operators
# operators_unique.drop_duplicates('unique', inplace=True)

In [485]:
#create a new dataframe and fill it with operators data
unique_ops = pd.DataFrame()
unique_ops = unique_ops.append(operators)

In [486]:
#drop the duplciates out of it 
unique_ops.drop_duplicates('unique', inplace=True)

In [487]:
#assign a numeric 'id' field
unique_ops['operator_id'] = unique_ops.index + 1

In [488]:
#and write the necessary fields into a csv
unique_ops[['operator_display_name', 'address1', 'address2', 'city', 'zipcode15', 'zipcode69', 'phone_number', 'release_text', 'operator_id']].to_csv('data/processed/operators.csv')

In [489]:
unique_ops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15327 entries, 0 to 17862
Data columns (total 14 columns):
id                       15327 non-null int64
auth_no                  15327 non-null object
operator_display_name    15327 non-null object
address1                 15327 non-null object
address2                 504 non-null object
city                     15327 non-null object
zipcode15                15323 non-null object
zipcode69                1383 non-null object
ST2                      15327 non-null object
phone_number             3952 non-null object
release_text             15327 non-null object
field_office_id          15327 non-null int64
unique                   15327 non-null object
operator_id              15327 non-null int64
dtypes: int64(3), object(11)
memory usage: 1.8+ MB


### Authorization numbers table
---
Create a table that links authorization numbers to operators.

In [490]:
authorizations = operators[['operator_display_name', 'address1', 'auth_no']]

In [491]:
authorizations['unique'] = authorizations['operator_display_name'] + authorizations['address1']
authorizations['id'] = authorizations.index + 1

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [492]:
# authorizations

In [493]:
#check the first fiew rows of the file
authorizations[:3]

,operator_display_name,address1,auth_no,unique,id
0,"NOEL,MICHAEL E,SHERRY M;STEWART,TREVOR D,JENNI...",P O BOX 301,0200054,"NOEL,MICHAEL E,SHERRY M;STEWART,TREVOR D,JENNI...",1
1,DUANE E. AND IVONA H. MAGOON,P.O. BOX 7070,0200064,DUANE E. AND IVONA H. MAGOONP.O. BOX 7070,2
2,"LARSON, L. LYNN AND SANDRA",213 E. 670 SOUTH,0200096,"LARSON, L. LYNN AND SANDRA213 E. 670 SOUTH",3


In [494]:
# write two columns into a csv that we can import to Django app
authorizations[['auth_no', 'id']].to_csv('data/processed/authorizations.csv')

### Linking authorization numbers to operators 

In [495]:
# operators_w_auth_id = pd.merge(operators, authorizations, on='auth_no', how='left')
# operators_w_auth_id.to_csv('data/processed/operators_w_auth_id.csv')
# operators.to_csv('data/processed/operators.csv')

In [496]:
unique_ops_short = unique_ops[['unique', 'operator_id']]
operators_w_id = pd.merge(operators, unique_ops_short, on='unique', how='left')

In [497]:
# check the new dataframe 
operators_w_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17863 entries, 0 to 17862
Data columns (total 14 columns):
id                       17863 non-null int64
auth_no                  17863 non-null object
operator_display_name    17863 non-null object
address1                 17863 non-null object
address2                 601 non-null object
city                     17863 non-null object
zipcode15                17859 non-null object
zipcode69                1628 non-null object
ST2                      17863 non-null object
phone_number             4938 non-null object
release_text             17863 non-null object
field_office_id          17863 non-null int64
unique                   17863 non-null object
operator_id              17863 non-null int64
dtypes: int64(3), object(11)
memory usage: 2.0+ MB


In [498]:
authorizations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17863 entries, 0 to 17862
Data columns (total 5 columns):
operator_display_name    17863 non-null object
address1                 17863 non-null object
auth_no                  17863 non-null object
unique                   17863 non-null object
id                       17863 non-null int64
dtypes: int64(1), object(4)
memory usage: 837.3+ KB


In [499]:
# now merge that new dataframe with authorizations 
auths = authorizations[['auth_no', 'id']]
auths = auths.rename(columns={'id': 'authorization_id'})
operators_w_id_w_auth = pd.merge(operators_w_id, auths, on = 'auth_no')

In [500]:
operators_w_id_w_auth['id'] = operators_w_id_w_auth.index + 1

In [501]:
operators_w_id_w_auth[['id', 'operator_id', 'authorization_id']].to_csv('data/processed/operator_auth_no.csv')

--------

### Allotments table
Create a table of operators linked to field offices and operators, along witha unique ID that can be referencecd by other tables such as "health" and "boundary"

In [502]:
#save a copy of all the states to a new csv just because
allotments.to_csv('data/rangeland-administration-system/allotment-info/all_states.csv')

In [503]:
#create a new column with unique id for allotments based on state and allotment number (allotment numbers are unique within states according to BLM documentation)
allotments['allotment_unique'] = allotments['Adm State'] + allotments['Allotment Number']

In [504]:
#check the first few lines to make sure we're all good
allotments["allotment_unique"][:5]
allotments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25307 entries, 0 to 4190
Data columns (total 16 columns):
Adm State                25307 non-null object
Admin Office             25307 non-null object
Field Office             25307 non-null object
Allotment Number         25307 non-null object
Allotment Name           25307 non-null object
Available For Grazing    25307 non-null object
Grazing Decision         122 non-null object
Public Acres             25215 non-null object
Amp Text                 6869 non-null object
Amp Implement Date       6759 non-null object
Management Stat Text     25307 non-null object
Auth No                  23743 non-null object
Permitted Aums           23629 non-null object
Suspended Aums           18466 non-null object
Susp Use Temp            11974 non-null object
allotment_unique         25307 non-null object
dtypes: object(16)
memory usage: 3.3+ MB


In [505]:
# create a new dataframe and drop the duplicate allotment numbers out of it
allotments_trimmed = allotments[['allotment_unique', 'Admin Office', 'Auth No', 'Allotment Name', 'Allotment Number', 'Available For Grazing', 'Grazing Decision', 'Public Acres', 'Amp Text', 'Amp Implement Date', 'Management Stat Text']]

In [506]:
# drop the duplicates out of it
allotments_trimmed.drop_duplicates('allotment_unique', inplace = True)

In [507]:
# rename columns to get rid of messy spaces and capitals
allotments_trimmed = allotments_trimmed.rename(columns={'Admin Office': 'office_code', 'Auth No': 'auth_no', 'Allotment Number': 'allotment_number', 'Allotment Name': 'allotment_name', 'Available For Grazing': 'available_for_grazing', 'Grazing Decision': 'grazing_decision', 'Public Acres': 'public_acres', 'Amp Text': 'amp_text', 'Amp Implement Date': 'amp_implement_date', 'Management Stat Text': 'management_stat_text'})

In [508]:
# check it out to make sure we're still good ... 
allotments_trimmed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21265 entries, 0 to 4189
Data columns (total 11 columns):
allotment_unique         21265 non-null object
office_code              21265 non-null object
auth_no                  19701 non-null object
allotment_name           21265 non-null object
allotment_number         21265 non-null object
available_for_grazing    21265 non-null object
grazing_decision         122 non-null object
public_acres             21205 non-null object
amp_text                 4820 non-null object
amp_implement_date       4739 non-null object
management_stat_text     21265 non-null object
dtypes: object(11)
memory usage: 1.9+ MB


In [509]:
# join it with field office
allotments_with_field_office = pd.merge(allotments_trimmed, field_offices, on='office_code', how='inner')

In [510]:
#strip the comma out of "public acres" and convert it to a float field so we can calc on it. Then describe() just to make sure it worked
allotments_with_field_office['public_acres'] = allotments_with_field_office['public_acres'].str.replace(',', '')
allotments_with_field_office['public_acres'] = allotments_with_field_office['public_acres'].astype(float)
allotments_with_field_office['public_acres'].describe()

count      21205.000000
mean        7286.288847
std        30588.340811
min            0.000000
25%          240.000000
50%          960.000000
75%         4091.000000
max      1459993.000000
Name: public_acres, dtype: float64

In [511]:
allotments_with_field_office = allotments_with_field_office.rename(columns={'id': 'field_office_id'})

In [512]:
#reassign the variable name to a data frame selecting only the fields we want for our table
allotments_with_field_office = allotments_with_field_office[['allotment_unique', 'allotment_number', 'allotment_name', 'available_for_grazing', 'grazing_decision', 'public_acres', 'amp_text', 'management_stat_text', 'field_office_id', 'amp_implement_date']]

In [681]:
#convert 'amp_implement_date' into a Django-friendly format, then check it with the first five rows jsut to make sure
allotments_with_field_office['amp_implement_date'] = pd.to_datetime(allotments_with_field_office['amp_implement_date'])

# fill the nulls with a common date that none of the actual dates will match and that we can filter out later
# this is to help avoid an error on postgres import
allotments_with_field_office['amp_implement_date'] = allotments_with_field_office['amp_implement_date'].fillna(1930-01-01)

allotments_with_field_office['amp_implement_date'].head()

0   1981-11-01 00:00:00.000000000
1   1970-01-01 00:00:00.000001928
2   1970-01-01 00:00:00.000001928
3   1983-08-30 00:00:00.000000000
4   1991-06-27 00:00:00.000000000
Name: amp_implement_date, dtype: datetime64[ns]

In [682]:
# assign each allotment a unique id
allotments_with_field_office['id'] = allotments_with_field_office.index + 1
allotments_with_field_office['id'].head()

0    1
1    2
2    3
3    4
4    5
Name: id, dtype: int64

In [515]:
allotments_with_field_office.to_csv('data/processed/allotments.csv')

----

### Permits table
Create a table of permits linked to operators and allotments and with a unique ID that can be referencecd by other tables

In [675]:
permits.head()

,office_code,Fo Name,Auth No,pl_effect_dt,pl_exp_dt,permit_status,allotment_number,Allotment Name,livestock_number,livestock_kind,pd_beg_dt,pd_end_dt,type_use,pl_percent,aums
0,LLAZA01000,ARIZONA STRIP FO,0200054,9/1/2007,8/30/2017,Blank,05316,LOST SPRING GAP,12,CATTLE,3/1/2003,4/30/2003,ACTIVE,100,24
1,LLAZA01000,ARIZONA STRIP FO,0200054,9/1/2007,8/30/2017,Blank,05316,LOST SPRING GAP,12,CATTLE,1/1/2003,2/28/2003,ACTIVE,100,23
2,LLAZA01000,ARIZONA STRIP FO,0200064,3/1/2015,2/28/2025,FLPMA 402(C)(2)/APPROP ACT,04810,SULLIVAN CANYON,72,CATTLE,3/1/2003,2/28/2004,ACTIVE,100,864
3,LLAZA01000,ARIZONA STRIP FO,0200096,3/1/2010,2/28/2017,FLPMA 402(C)(2)/APPROP ACT,04842,CEDAR WASH,67,CATTLE,10/16/2004,2/28/2005,ACTIVE,100,300
4,LLAZA01000,ARIZONA STRIP FO,0200096,3/1/2010,2/28/2017,FLPMA 402(C)(2)/APPROP ACT,04842,CEDAR WASH,67,CATTLE,3/1/2005,3/15/2005,ACTIVE,100,33


In [517]:
permits = permits.rename(columns={'Off CD': 'office_code', 'P/L Eff Dt': 'pl_effect_dt', 'P/L exp Dt': 'pl_exp_dt', 'Permit Status': 'permit_status', 'Allotment Number': 'allotment_number', 'Lvsk #': 'livestock_number', 'Lvsk Kind': 'livestock_kind', 'Pd Beg Dt': 'pd_beg_dt', 'Pd End Dt': 'pd_end_dt', 'Type Use': 'type_use', 'PL %': 'pl_percent', 'Aums': 'aums'} )

In [518]:
permits_fo = pd.merge(permits, field_offices, on='office_code', how='inner')

In [519]:
permits_fo = permits_fo.rename(columns={'id': 'field_office_id'})

In [520]:
#convert date fields to datetime
permits_fo['pl_effect_dt'] = pd.to_datetime(permits_fo['pl_effect_dt'])
permits_fo['pl_exp_dt'] = pd.to_datetime(permits_fo['pl_exp_dt'])

permits_fo['pd_beg_dt'] = pd.to_datetime(permits_fo['pd_beg_dt'])
permits_fo['pd_end_dt'] = pd.to_datetime(permits_fo['pd_end_dt'])

In [676]:

permits_fo.head()

,auth_no,pl_effect_dt,pl_exp_dt,permit_status,allotment_number,livestock_number,livestock_kind,pd_beg_dt,pd_end_dt,type_use,pl_percent,aums,field_office_id,allotment_unique
0,0200054,2007-09-01,2017-08-30,NaN,05316,12,CATTLE,2003-03-01,2003-04-30,ACTIVE,100,24,1,AZ05316
1,0200054,2007-09-01,2017-08-30,NaN,05316,12,CATTLE,2003-01-01,2003-02-28,ACTIVE,100,23,1,AZ05316
2,0200064,2015-03-01,2025-02-28,FLPMA 402(C)(2)/APPROP ACT,04810,72,CATTLE,2003-03-01,2004-02-28,ACTIVE,100,864,1,AZ04810
3,0200096,2010-03-01,2017-02-28,FLPMA 402(C)(2)/APPROP ACT,04842,67,CATTLE,2004-10-16,2005-02-28,ACTIVE,100,300,1,AZ04842
4,0200096,2010-03-01,2017-02-28,FLPMA 402(C)(2)/APPROP ACT,04842,67,CATTLE,2005-03-01,2005-03-15,ACTIVE,100,33,1,AZ04842


In [522]:
#strip the two letter state abbr out of the office code
permits_fo['state'] = permits_fo['office_code'].str[2:4]

In [523]:
#create the unique allotment id by concatenating state abbr with allotment number
permits_fo['allotment_unique'] = permits_fo['state'] + permits_fo['allotment_number']

In [677]:
#check to see we got it right 
permits_fo['allotment_unique'].head()

0    AZ05316
1    AZ05316
2    AZ04810
3    AZ04842
4    AZ04842
Name: allotment_unique, dtype: object

In [525]:
permits_fo = permits_fo.rename(columns={'Auth No': 'auth_no'})

In [526]:
permits_fo =  permits_fo[['auth_no', 'pl_effect_dt', 'pl_exp_dt', 'permit_status', 'allotment_number', 'livestock_number', 'livestock_kind', 'pd_beg_dt', 'pd_end_dt', 'type_use', 'pl_percent', 'aums', 'field_office_id', 'allotment_unique']]

In [678]:
permits_fo.head()

,auth_no,pl_effect_dt,pl_exp_dt,permit_status,allotment_number,livestock_number,livestock_kind,pd_beg_dt,pd_end_dt,type_use,pl_percent,aums,field_office_id,allotment_unique
0,0200054,2007-09-01,2017-08-30,NaN,05316,12,CATTLE,2003-03-01,2003-04-30,ACTIVE,100,24,1,AZ05316
1,0200054,2007-09-01,2017-08-30,NaN,05316,12,CATTLE,2003-01-01,2003-02-28,ACTIVE,100,23,1,AZ05316
2,0200064,2015-03-01,2025-02-28,FLPMA 402(C)(2)/APPROP ACT,04810,72,CATTLE,2003-03-01,2004-02-28,ACTIVE,100,864,1,AZ04810
3,0200096,2010-03-01,2017-02-28,FLPMA 402(C)(2)/APPROP ACT,04842,67,CATTLE,2004-10-16,2005-02-28,ACTIVE,100,300,1,AZ04842
4,0200096,2010-03-01,2017-02-28,FLPMA 402(C)(2)/APPROP ACT,04842,67,CATTLE,2005-03-01,2005-03-15,ACTIVE,100,33,1,AZ04842


In [528]:
# merge permits with allotments`
permits_fo_allot = pd.merge(permits_fo, allotments_with_field_office, on='allotment_unique', how='inner')

In [529]:
# rename some key columns
permits_fo_allot = permits_fo_allot.rename(columns={'id': 'allotment_id', 'field_office_id_x': 'field_office_id', 'auth_no_x': 'auth_no'})

In [530]:
# select only the columns we need
permits_fo_allot = permits_fo_allot[['auth_no', 'pl_effect_dt', 'pl_exp_dt', 'permit_status', 'livestock_number', 'livestock_kind', 'pd_beg_dt', 'pd_end_dt', 'type_use', 'pl_percent', 'aums', 'field_office_id', 'allotment_unique', 'allotment_id']]

In [531]:
permits_fo_allot_auths = pd.merge(permits_fo_allot, auths, on='auth_no', how='inner')

In [532]:
permits_fo_allot_auths = permits_fo_allot_auths.rename(columns={'authorization_id': 'auth_no_id', 'field_office_id_x': 'field_office_id'})

In [533]:
permits_fo_allot_auths = permits_fo_allot_auths[['auth_no', 'pl_effect_dt', 'pl_exp_dt', 'permit_status', 'livestock_number', 'livestock_kind', 'pd_beg_dt', 'pd_end_dt', 'type_use', 'pl_percent', 'aums', 'field_office_id', 'allotment_unique', 'allotment_id', 'auth_no_id']]

In [534]:
permits_fo_allot_auths['id'] = permits_fo_allot_auths.index + 1
permits_fo_allot_auths[:2]

,auth_no,pl_effect_dt,pl_exp_dt,permit_status,livestock_number,livestock_kind,pd_beg_dt,pd_end_dt,type_use,pl_percent,aums,field_office_id,allotment_unique,allotment_id,auth_no_id,id
0,0200054,2007-09-01,2017-08-30,NaN,12,CATTLE,2003-03-01,2003-04-30,ACTIVE,100,24,1,AZ05316,99,1,1
1,0200054,2007-09-01,2017-08-30,NaN,12,CATTLE,2003-01-01,2003-02-28,ACTIVE,100,23,1,AZ05316,99,1,2


In [535]:
permits_fo_allot_auths['pl_effect_dt'] = permits_fo_allot_auths['pl_effect_dt'].fillna(1930-01-01)
permits_fo_allot_auths['pl_exp_dt'] = permits_fo_allot_auths['pl_exp_dt'].fillna(1930-01-01)
permits_fo_allot_auths['pd_beg_dt'] = permits_fo_allot_auths['pd_beg_dt'].fillna(1930-01-01)
permits_fo_allot_auths['pd_end_dt'] = permits_fo_allot_auths['pd_end_dt'].fillna(1930-01-01)

In [536]:
permits_fo_allot_auths.to_csv('data/processed/permits.csv')

### Allotments and authorizations many-to-many table
An outer join of allotment ids, numbers and operator ids, numbers for the Many to Many relationship between allotments and operators

In [537]:
allotments = allotments.rename(columns={'Auth No': 'auth_no'})
allotments_operators = pd.merge(auths, allotments, on="auth_no", how='left')
#allotments_operators = allotments_operators.rename(columns={'id': 'operator_id'})

In [538]:
allotments_operators = pd.merge(allotments_operators, allotments_with_field_office, on='allotment_unique')

In [539]:
allotments_operators = allotments_operators.rename(columns={'id': 'allotment_id'})

In [540]:
allotments_operators = allotments_operators[['allotment_id', 'authorization_id']]

In [541]:
allotments_operators['id'] = allotments_operators.index + 1

In [542]:
allotments_operators.to_csv('data/processed/allotments_auth_no.csv')

-------

### Health table

The health table is in shapefile format, provided by Peter Lattin, landscape ecologist and former BLM contractor, who cleaned and analyzed the data for Public Employees for Environmental Responsibility. 

See rangeland.grazing.load for how this shapefile is handled.

----

# Analysis for story
---

### Allotment management plans
Finding the total number / percent of allotments lacking an allotment management plan (AMP), and the total number / percent of allotments with an AMP that is more than 10, 20 or 30 years old.

In [543]:
# fill the amp_text nulls with a value so that it can be included in a groupby 
allots = allotments_with_field_office
allots['amp_text'] = allots['amp_text'].fillna('None')

In [627]:
#allots.info()

The count of allotment management plans. 
Note that roughly 5,000 allotments listed have some form of allotment management plan proposed or implemented, and more than 16,000 allotments have nothing listed.

In [631]:
allots.groupby('amp_text').count().loc[:,'id']

amp_text
AMP IMPLEMENTED         4516
AMP PROPOSED              79
AMP WRITTEN               46
AMP/CMP IMPLEMENTED       39
CMP IMPLEMENTED          140
None                   16445
Name: id, dtype: int64

The total number / percent of permits renewed under the appropriations act, without an environmental analysis:

In [653]:
# send it to a new dataframe
by_amp = pd.DataFrame({
        "public_acres": allots.groupby('amp_text').sum().loc[:,'public_acres'], 
        "number_of_allotments": allots.groupby('amp_text').count().loc[:,'id']
    })

print 'Grouped by AMP status. "None" means blank entry in BLM data.'
by_amp

Grouped by AMP status. "None" means blank entry in BLM data.


,number_of_allotments,public_acres
amp_text,,
AMP IMPLEMENTED,4516,80316572
AMP PROPOSED,79,1018533
AMP WRITTEN,46,414191
AMP/CMP IMPLEMENTED,39,1224730
CMP IMPLEMENTED,140,1915468
None,16445,69616261


The percent of acres BLM allots for grazing that do not have an allotment management plan:

In [633]:
acres_without_amps = by_amp['public_acres'][5:6].sum() / by_amp['public_acres'].sum()
allotments_without_amps =  by_amp['number_of_allotments'][5:6].sum() / by_amp['number_of_allotments'].sum()

print "About {} percent of allotments, {} in total, do not have an allotment management plan listed. This represents roughly {} percent of the total number of acres.".format(round(allotments_without_amps * 100), by_amp['number_of_allotments'][5:6].sum(), round(acres_without_amps * 100))

About 77.0 percent of allotments, 16445 in total, do not have an allotment management plan listed. This represents roughly 45.0 percent of the total number of acres.


### Appropriations Act permit approval
The number and percent of permits renewed under the appropriations act (without an environmental analysis).

In [672]:
#create a new df we'll use for the analysis by compying permits dataframe
permits_for_analysis = permits

#fill the blanks with a string so we can use it as a group by category
permits_for_analysis['permit_status'] = permits_for_analysis['permit_status'].fillna('Blank')

#create a new data frame with just the info we need for our analysis
by_permit = pd.DataFrame({
        "number_of_permits": permits_for_analysis.groupby('permit_status').count().loc[:,'allotment_number']
    })

#and print that dataframe
print "Grouped by the status of the permit. Note the nubmer of permits approved under FLPMA 402(C)(2)/Appropriations Act"
by_permit

Grouped by the status of the permit. Note the nubmer of permits approved under FLPMA 402(C)(2)/Appropriations Act


,number_of_permits
permit_status,
Blank,20039
DECISION-STAYED,97
FLPMA 402(C)(2)/APPROP ACT,13473
HOLD,328


** Percent of permits issued without an environmental assessment **

In [673]:
pct_appropriations_act = by_permit['number_of_permits'][2:3].sum() / by_permit['number_of_permits'].sum()

"Roughly {} percent of existing permits were issued under the Appropriations Act rider that allows permits to be renewed for 10 years as-is without review.".format(round(pct_appropriations_act * 100))

'Roughly 40.0 percent of existing permits were issued under the Appropriations Act rider that allows permits to be renewed for 10 years as-is without review.'

### Further questions:
* How many permits issued under NEPA exemption were on allotments that do not meet health standard or are considered "needing improvement"?
* How many allotments in the needing improvement category are missing an allotment management plan? 